In [10]:
import os

from pyspark.sql import SparkSession
from pyspark.sql.utils import AnalysisException

spark = SparkSession \
    .builder.appName("test spark datasource generic file source option") \
    .master("local[*]") \
    .getOrCreate()
# These generic options/configurations are effective only when using file-based sources: parquet, orc, avro, json, csv, text.
# Please note that the hierarchy of directories used in examples below are:
# dir1/
# ├── dir2/
# │    └── file2.parquet (schema: <file: string>, content: "file2.parquet")
# └── file1.parquet (schema: <file, string>, content: "file1.parquet")
# └── file3.json (schema: <file, string>, content: "{'file':'corrupt.json'}")

## Ignore Corrupt Files
# Spark allows you to use spark.sql.files.ignoreCorruptFiles to ignore corrupt files while reading data from files. When set to true, the Spark jobs will continue to run when encountering corrupted files and the contents that have been read will still be returned.

# enable ignore corrupt files
spark.sql("set spark.sql.files.ignoreCorruptFiles=true")

# dir1/file3.json is corrupt from parquet's view
test_corrupt_df = spark.read.parquet("resources/dir1","resources/dir1/dir2")
test_corrupt_df.show()


+-------------+
|         file|
+-------------+
|file1.parquet|
|file2.parquet|
|file2.parquet|
+-------------+



In [ ]:
## Ignore Missing Files
# Spark allows you to use spark.sql.files.ignoreMissingFiles to ignore missing files while reading data from files. Here, missing file really means the deleted file under directory after you construct the DataFrame. When set to true, the Spark jobs will continue to run when encountering missing files and the contents that have been read will still be returned.

In [4]:
## Path Global Filter
# pathGlobFilter is used to only include files with file names matching the pattern. The syntax follows org.apache.hadoop.fs.GlobFilter. It does not change the behavior of partition discovery.
# To load files with paths matching a given glob pattern while keeping the behavior of partition discovery, you can use:
spark.sql("set spark.sql.files.ignoreCorruptFiles=false")
df = spark.read.load("resources/dir1/",format="parquet",pathGlobFilter="*.parquet")
df.show()

+-------------+
|         file|
+-------------+
|file1.parquet|
+-------------+



In [14]:
## Recursive File Lookup
# recursiveFileLookup is used to recursively load files and it disables partition inferring. Its default value is false. If data source explicitly specifies the partitionSpec when recursiveFileLookup is true, exception will be thrown.
recursive_load_df = spark.read.format("parquet") \
    .option("recursiveFileLookup","true") \
    .option("pathGlobFilter","*.parquet") \
    .load("resources/dir1/")
recursive_load_df.show()



+-------------+
|         file|
+-------------+
|file1.parquet|
|file2.parquet|
+-------------+



In [8]:
## Modification Time Path Filters
# modifiedBefore and modifiedAfter are options that can be applied together or separately in order to achieve greater granularity over which files may load during a Spark batch query. (Note that Structured Streaming file sources don’t support these options.)
# modifiedBefore: an optional timestamp to only include files with modification times occurring before the specified time. The provided timestamp must be in the following format: YYYY-MM-DDTHH:mm:ss (e.g. 2020-06-01T13:00:00)
# modifiedAfter: an optional timestamp to only include files with modification times occurring after the specified time. The provided timestamp must be in the following format: YYYY-MM-DDTHH:mm:ss (e.g. 2020-06-01T13:00:00)
# When a timezone option is not provided, the timestamps will be interpreted according to the Spark session timezone (spark.sql.session.timeZone).
# To load files with paths matching a given modified time range, you can use:

# Only load files modified before 07/1/2050 @ 08:30:00
df = spark.read.load("resources/dir1",
                     format="parquet"
                     ,pathGlobFilter="*.parquet"
                     , modifiedBefore="2050-07-01T08:30:00")
df.show()

# Only load files modified after 06/01/2050 @ 08:30:00
try:
    df = spark.read.load("resources/dir1",
                         format="parquet"
                         ,pathGlobFilter="*.parquet"
                         , modifiedAfter="2022-02-10T18:10:00")
    df.show()
#     "2050-06-01T08:30:00"
except AnalysisException as ex:
    print("dataframe is empty",ex.desc)



+-------------+
|         file|
+-------------+
|file1.parquet|
|file2.parquet|
+-------------+

+-------------+
|         file|
+-------------+
|file1.parquet|
|file2.parquet|
+-------------+



In [17]:
def getFilesModifiedAfter(path, modifiedAfter):
    f=[]
    for root, dirs, files in os.walk(path):
        # newfiles=filter(lambda file:os.stat(root+"/"+file).st_mtime>modifiedAfter , files)
        for file in files:
            if os.stat(root+"/"+file).st_mtime>modifiedAfter:
                f.append(root+"/"+file)
    return f

getFilesModifiedAfter("resources/dir1",1644056071.685726)


['resources/dir1/file1.parquet',
 'resources/dir1/file2.parquet',
 'resources/dir1/file3.json',
 'resources/dir1\\dir2/file2.parquet']

In [19]:
try:
    df = spark.read.load(getFilesModifiedAfter("resources/dir1",1644856071.685726),
                         format="parquet"
                         ,pathGlobFilter="*.parquet"
                         , modifiedAfter="2022-02-10T18:10:00")
    df.show()
#     "2050-06-01T08:30:00"
except AnalysisException as ex:
    print("dataframe is empty",ex.desc)


dataframe is empty Unable to infer schema for Parquet. It must be specified manually.
